# Sortino ratio optimisation with risk budgetting

The objective of this program is to maximise the Sortino ratio of the portfolio such that all stocks have a weight allocation of at least min_weight and a maximum weight allocation of max_weight. Moreover, each stock cannot have more than an x percentage contribution to the total portfolio volatility. 

The package scipy.optimize can only minimise the objective function. Hence, the negative Sortino Ratio is minimised. 

## Definition Sortino ratio
The Sortino ratio is a measure that uses a risk-adjusted return measure based on downside risk. This differs from the Sharpe ratio, as this measure makes use of the total (positive and negative) volatility. The Sortino ratio is calculated in the following way:

$ \text{Sortino ratio} = \frac{R_p-B}{\sigma_d} $ \
 Where $R_p$: portfolio return \
$\hspace{1cm}$  $B$: benchmark \
$\hspace{1cm}$  $\sigma_d$: downside risk of the portfolio. 



## Downside risk // semi deviation w.r.t. B

Variance of a stock A: $\sigma^2_A=\frac{1}{T} \sum_{t=1}^T (min\{R^{A}_t-B,0\})^2$

Covariance of stocks A and C: $\sigma_A\sigma_C=\frac{1}{T} \sum_{t=1}^T min\{R^{A}_t-B,0\}*min\{R^{C}_t-B,0\}$

In [160]:
import pandas as pd   
import numpy as np
import matplotlib.pyplot as plt
import pandas_datareader as dr 
from datetime import datetime
from scipy.optimize import minimize 

In [161]:
# packages installation anaconda + version used
 # conda install pandas=1.2.2
 # conda install pandas-datareader=0.9.0
 # conda install scipy=1.6.1
 # conda install openpyxl=3.0.6

### Predefined values

In [162]:
benchmark = 0.01
min_weight = 0.025
max_weight = 1
max_risk_contribution = 0.40

### Data preprocessing 

In [163]:
# Initialize Input Data
start_date = '2017-01-01'
today = datetime.today().strftime('%Y-%m-%d')
#assets = pd.read_excel ('SigmaInput.xlsx')["Ticker"]
assets = ["AD.AS", "BABA","PVH",]

# Create df for adjusted close prices of portfolio
prices = pd.DataFrame()

for stock in assets:
    prices[stock] = dr.data.get_data_yahoo(stock, start =start_date, end =today)['Adj Close']

#Show the daily simple return
returns = (prices/prices.shift(1))-1

#return_stocks.to_excel('PortfolioReturns.xlsx')

In [164]:
# Substract the benchmark returns from the stock returns and take the min{Rt-B, 0}
step1 = returns - benchmark
step1[step1 > 0] = 0

In [165]:
step1

,AD.AS,BABA,PVH
Date,,,
2017-01-02,NaN,NaN,NaN
2017-01-03,-0.015729,NaN,NaN
2017-01-04,-0.015261,0.000000,0.000000
2017-01-05,-0.020073,0.000000,0.000000
2017-01-06,-0.012035,-0.015086,-0.011076
...,...,...,...
2021-03-05,0.000000,0.000000,0.000000
2021-03-08,-0.009555,-0.040784,0.000000
2021-03-09,-0.005556,0.000000,-0.034543


In [166]:
cov_matrix = covarianceMatrix(assets, step1)
print(cov_matrix)

[[0.05928111 0.03967981 0.04962909]
 [0.03967981 0.10798102 0.07905007]
 [0.04962909 0.07905007 0.18740431]]


### Functions 

In [167]:
def covariance(returnsA, returnsB):
    # Multiply min{R_{ta}-B, 0}*min{R_{tc}-B, 0}
    multiplication = returnsA*returnsB  
    # Obtain T
    t = multiplication.count()
    # Sum over all t = {1,2,...,T}
    summation = multiplication.sum(axis=0)
    # Calculate downside variance and convert them to yearly values
    downside_variance = 1/(t)*summation*252
    return downside_variance

In [168]:
def covarianceMatrix(assetlist, step1data):
    covariance_matrix = np.zeros((len(assetlist), len(assetlist)))
    
    # loop for indices [0,n-2] -> all elements except [0][n-1], [n-1][0], [n-1][n-1], where n = len(assets)
    for x in range(len(assetlist)-1):
        # variance
        covariance_matrix[x][x] = covariance(step1data[assetlist[x]], step1data[assetlist[x]])
        # covariance
        value = covariance(step1data[assetlist[x]], step1data[assetlist[x+1]])
        covariance_matrix[x][x+1] = value
        covariance_matrix[x+1][x] = value
    
    # Set missing values
    index = len(assetlist) - 1
    covariance_matrix[index][index] = covariance(step1data[assetlist[index]], step1data[assetlist[index]])
    value = covariance(step1data[assetlist[index]], step1data[assetlist[0]])
    covariance_matrix[0][index] = value
    covariance_matrix[index][0] = value

    return(covariance_matrix)

In [169]:
# function to obtain portfolio-{return,volatility,sortino}
def get_p_data(weights):
        weights = np.array(weights)
        p_return= np.sum(returns.mean()*weights) * 252 # Taken from the positive and negative returns -> still multiply  by 252
        p_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) # Takes the downside risk as a risk measure -> already multiplied by 252
        p_sortino = (p_return-benchmark)/p_volatility 
        return np.array([p_return, p_volatility, p_sortino])

def get_p_volatility(weights):
        weights = np.array(weights)
        p_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
        return p_volatility

In [170]:
# function to get the marginal volatilities 
def marginal_volatilities(weights, cov_matrix_np):
    volatility_p = get_p_volatility(weights)
    print("portfolio volatility = ", volatility_p )
        
    for i in range(len(weights)):
        cov_weight_i = (weights[i] * np.dot(cov_matrix_np, weights)[i])/volatility_p 
        print("i = ", i , "risk = ", cov_weight_i, "  % = ", (cov_weight_i/volatility_p))

In [171]:
# function to get negative Sortino Ratio.
def negative_sortino(weights):
        return get_p_data(weights)[2] * -1

# function to check if sum investment is 1.
def check_sum(weights):
        return np.sum(weights) -1

In [172]:
# args is defined as the tuple of arguments with first the max risk contribution and then the covariance matrix
def risk_budget(weights, i, *args):
    volatility_p = get_p_volatility(weights)
    dot_product = np.dot(args[1], weights)
    dot_product_value = np.take(dot_product, i)
    q = args[0]*volatility_p - (weights[i]*dot_product_value/volatility_p)
    return q

### Optimisation

In [173]:
# constraints
constraints_list = list()

#  sum investment is 1
constraints_list.append({'type':'eq', 'fun':check_sum})

# individual risk constributions
a = list(range(len(assets)))
for i in a:
    constraints_list.append({'type':'ineq', 'fun': risk_budget, 'args':(i, max_risk_contribution, cov_matrix)})  

constraints = tuple(constraints_list)

In [174]:
# Weight boundaries
bounds = ((min_weight,max_weight),)*len(assets)

# Equally weighted portfolio / starting point of Sequantial least Squares programming method.
equally_weighted = [1/len(assets),]*len(assets)

In [175]:
# Optimisation method and results
optimal_result = minimize(negative_sortino, equally_weighted, method='SLSQP', bounds=bounds, constraints = constraints)
optimal_weights = optimal_result.x
optimal_portfolio = get_p_data(optimal_result.x)

optimal_return = np.round(optimal_portfolio[0],2)
optimal_volatility = np.round(optimal_portfolio[1],2)
optimal_sortino = np.round(optimal_portfolio[2],2)
overview = pd.DataFrame({'Assets': assets,'Weight (%)': np.round(optimal_weights*100,2)}, columns=['Assets', 'Weight (%)']).T

statistic = ['Number of Assets', 'Expected Return', 'Expected Volatility', 'Expected Sortino']
data = [np.round(len(assets),0), optimal_return, optimal_volatility, optimal_sortino]
table = pd.DataFrame(data, statistic)

In [176]:
# Results
print(overview)
print(table)

                0     1      2
Assets      AD.AS  BABA    PVH
Weight (%)  31.21  39.0  29.79
                        0
Number of Assets     3.00
Expected Return      0.20
Expected Volatility  0.28
Expected Sortino     0.70


In [177]:
marginal_volatilities(optimal_weights, cov_matrix)

portfolio volatility =  0.275841622820156
i =  0 risk =  0.05516836215486263   % =  0.2000001362768644
i =  1 risk =  0.11033666831336109   % =  0.40000006955186274
i =  2 risk =  0.11033659235193224   % =  0.39999979417127274
